In [1]:
# This note book contains the code for performing iteravtive pruning on trained Resnet18 model on CIFAR 10 dataset

In [2]:
import torch
import copy
import torch.nn.utils.prune as prune
import math

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import time
from torchvision import datasets, transforms
import requests
from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import random
from sklearn.metrics import confusion_matrix

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = '/content/drive/MyDrive/cnn-pruning-status_200'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
# %cd $FOLDERNAME/cs682/datasets/
# !bash get_datasets.sh
%cd $FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/cnn-pruning-status_200


In [4]:
# Load the pretrained model
from cifar10_models.resnet import resnet18
my_model = resnet18()

# Pretrained model
my_model = resnet18(pretrained=True)
my_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
# Dataset Preprocessing
print("==> Preparing data..")
transform_train = transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010)),
])

==> Preparing data..


In [6]:
# Data augmentation and normalization for testing
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010)),
])

In [7]:
# Download the CIFAR10 dataset
trainset = torchvision.datasets.CIFAR10(
root='./data',train=True,download=True,transform=transform_train)

Files already downloaded and verified


In [8]:
torchloader = torch.utils.data.DataLoader(
trainset,batch_size=128,shuffle=True,num_workers=2)

In [9]:
testset = torchvision.datasets.CIFAR10(
root='./data',train=False,download=True,transform=transform_test)

Files already downloaded and verified


In [10]:
testloader = torch.utils.data.DataLoader(testset,batch_size=100,shuffle=False,num_workers=2)

In [11]:
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [12]:
epoch= 1
criterion = nn.CrossEntropyLoss()

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
#calculate sparsity ratio
def calculate_sparsity(model):
  total_count = 0
  zero_count = 0
  
  for buffer_name, buffer in model.named_buffers():
    zero_count += torch.sum(buffer == 0).item()
    if zero_count>0:
      total_count += buffer.nelement()

  print("Total params: ", total_count)
  print("Zero params: ", zero_count)
  return (math.ceil(zero_count*100/total_count))

In [ ]:
# Build and save the iterative pruning model with 50 % sparsity ratio
model1 = copy.deepcopy(my_model)
parameters_to_prune =[]
for module_name, module in model1.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, "weight"))
print("Layers to prune {}".format((len(parameters_to_prune))))
for i in range(5):
  prune.global_unstructured(
  parameters_to_prune,
  pruning_method=prune.L1Unstructured,
  amount=0.128,
  )
print("Sparsity of model1", calculate_sparsity(model1))
#torch.save(model1, "cifar10_models/state_dicts/Iterative_50.pt")

Layers to prune 21


In [ ]:
model1.to(device)

In [ ]:
# Test the accuracy of the model on the validation set
def test(epochs):
    model1.eval()
    val_acc=0
    val_loss=0
    val_correct=0
    val_total = 0
    with torch.no_grad():
        for batch_idx, (inputs,targets) in enumerate(testloader):
            inputs,targets = inputs.to(device),targets.to(device)
            outputs = model1(inputs)
            loss = criterion(outputs,targets)
            val_loss += loss.item()
            _,predicted = outputs.max(1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()
            val_acc = 100.*val_correct/val_total
            if batch_idx%40==39:
                print("Evaluating....")
                print(batch_idx,len(testloader),'Loss: %.3f | Acc: %0.3f (%d/%d)' % (val_loss/(batch_idx+1),100*val_correct/val_total,val_correct,val_total))
    

In [ ]:
# The model achieves an accuracy of ~93% on the test set.
for e in range(epoch):
    print("Epoch", e+1)
    test(epoch)

In [ ]:
#save the model
torch.save(model1, "cifar10_models/state_dicts/Iterative_50.pt")

In [ ]:
# Build and save the iterative pruning model with 75 % sparsity ratio
model2 = copy.deepcopy(my_model)
parameters_to_prune =[]
for module_name, module in model2.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, "weight"))
print("Layers to prune {}".format((len(parameters_to_prune))))
for i in range(5):
  prune.global_unstructured(
  parameters_to_prune,
  pruning_method=prune.L1Unstructured,
  amount=0.24,
  )
print("Sparsity of model2", calculate_sparsity(model2))


In [ ]:
model2.to(device)

In [ ]:
# Test the accuracy of the model on the validation set
def test(epochs):
    model2.eval()
    val_acc=0
    val_loss=0
    val_correct=0
    val_total = 0
    with torch.no_grad():
        for batch_idx, (inputs,targets) in enumerate(testloader):
            inputs,targets = inputs.to(device),targets.to(device)
            outputs = model2(inputs)
            loss = criterion(outputs,targets)
            val_loss += loss.item()
            _,predicted = outputs.max(1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()
            val_acc = 100.*val_correct/val_total
            if batch_idx%40==39:
                print("Evaluating....")
                print(batch_idx,len(testloader),'Loss: %.3f | Acc: %0.3f (%d/%d)' % (val_loss/(batch_idx+1),100*val_correct/val_total,val_correct,val_total))

    

In [ ]:
for e in range(epoch):
    print("Epoch", e+1)
    test(epoch)

In [ ]:
#save the model
torch.save(model2, "cifar10_models/state_dicts/Iterative_75.pt")

In [ ]:
# Build and save the iterative pruning model with 90 % sparsity ratio
model3 = copy.deepcopy(my_model)
parameters_to_prune =[]
for module_name, module in model3.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, "weight"))
print("Layers to prune {}".format((len(parameters_to_prune))))
for i in range(5):
  prune.global_unstructured(
  parameters_to_prune,
  pruning_method=prune.L1Unstructured,
  amount=0.36,
  )
print("Sparsity of model3", calculate_sparsity(model3))

In [ ]:
model3.to(device)

In [ ]:
# Test the accuracy of the model on the validation set
def test(epochs):
    model3.eval()
    val_acc=0
    val_loss=0
    val_correct=0
    val_total = 0
    with torch.no_grad():
        for batch_idx, (inputs,targets) in enumerate(testloader):
            inputs,targets = inputs.to(device),targets.to(device)
            outputs = model3(inputs)
            loss = criterion(outputs,targets)
            val_loss += loss.item()
            _,predicted = outputs.max(1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()
            val_acc = 100.*val_correct/val_total
            if batch_idx%40==39:
                print("Evaluating....")
                print(batch_idx,len(testloader),'Loss: %.3f | Acc: %0.3f (%d/%d)' % (val_loss/(batch_idx+1),100*val_correct/val_total,val_correct,val_total))
                
    

In [ ]:
for e in range(epoch):
    print("Epoch", e+1)
    test(epoch)

In [ ]:
#save the model
torch.save(model3, "cifar10_models/state_dicts/Iterative_90.pt")